In [1]:
!export TA_INCLUDE_PATH="$(brew --prefix ta-lib)/include"
!export TA_LIBRARY_PATH="$(brew --prefix ta-lib)/lib"
%arch -arm64 pip install TA-Lib

UsageError: Line magic function `%arch` not found.


In [4]:
# filepath: /Users/hemank/Documents/github/trading/backtesting/study.ipynb
import os
# Use actual paths instead of shell expressions
include_path = os.popen("brew --prefix ta-lib").read().strip() + "/include"
lib_path = os.popen("brew --prefix ta-lib").read().strip() + "/lib"

os.environ['TA_INCLUDE_PATH'] = include_path
os.environ['TA_LIBRARY_PATH'] = lib_path

import talib
print(talib.get_functions())

ImportError: dlopen(/Users/hemank/Documents/github/trading/ta-lib-python/talib/_ta_lib.cpython-311-darwin.so, 0x0002): tried: '/Users/hemank/Documents/github/trading/ta-lib-python/talib/_ta_lib.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/hemank/Documents/github/trading/ta-lib-python/talib/_ta_lib.cpython-311-darwin.so' (no such file), '/Users/hemank/Documents/github/trading/ta-lib-python/talib/_ta_lib.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))

In [54]:
# ----------- INSTALL DEPENDENCIES --------------

import backtrader as bt
import yfinance as yf
import numpy as np
import pandas as pd
import talib
# ----------- RSI STRATEGY USING TA-LIB --------------------
class RSIStrategy(bt.Strategy):
    params = dict(
        rsi_period=14,        # RSI calculation period
        rsi_low=30,           # Oversold threshold
        rsi_high=70,          # Overbought threshold
        printlog=True         # Enable trade logging
    )

    def __init__(self):
        # Convert backtrader data to numpy array for TA-Lib
        self.dataclose = self.datas[0].close
        
        # We'll calculate RSI in next() method since TA-Lib needs numpy arrays
        self.order = None

    def log(self, txt, dt=None):
        if self.params.printlog:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()} - {txt}')

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED: Price {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED: Price {order.executed.price:.2f}')
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Failed')
        self.order = None

    def notify_trade(self, trade):
        if trade.isclosed:
            self.log(f'TRADE CLOSED: PnL {trade.pnlcomm:.2f}')

    def next(self):
        # Skip if we have pending orders
        if self.order:
            return

        # Need at least RSI period + 1 bars for calculation
        if len(self) < self.params.rsi_period + 1:
            return

        # Get price data as numpy array for TA-Lib
        closes = np.array([self.dataclose[-i] for i in range(self.params.rsi_period + 10, -1, -1)])
        
        # Calculate RSI using TA-Lib
        rsi_values = talib.RSI(closes, timeperiod=self.params.rsi_period)
        current_rsi = rsi_values[-1]

        # Skip if RSI calculation returned NaN
        if np.isnan(current_rsi):
            return

        self.log(f'Price: {self.dataclose[0]:.2f}, RSI: {current_rsi:.2f}')

        # Trading Logic
        if not self.position:  # No position
            if current_rsi < self.params.rsi_low:  # Oversold
                self.log(f'BUY SIGNAL - RSI: {current_rsi:.2f}')
                self.order = self.buy()
            elif current_rsi > self.params.rsi_high:  # Overbought
                self.log(f'SELL SIGNAL - RSI: {current_rsi:.2f}')
                self.order = self.sell()
        
        else:  # Have position
            if self.position.size > 0:  # Long position
                if current_rsi > self.params.rsi_high:  # Exit long
                    self.log(f'CLOSE LONG - RSI: {current_rsi:.2f}')
                    self.order = self.close()
            elif self.position.size < 0:  # Short position
                if current_rsi < self.params.rsi_low:  # Exit short
                    self.log(f'CLOSE SHORT - RSI: {current_rsi:.2f}')
                    self.order = self.close()

# ----------- DATA DOWNLOAD & BACKTEST ---------------------
if __name__ == '__main__':
    # Download data
    ticker = "HDFCBANK.NS"
    df = yf.download(ticker, period="60d", interval="5m", auto_adjust=True, progress=False)

    # if df.empty:
    #     print(f"Error: No data for {ticker}")
    #     exit()

    # # Create backtrader data feed
    # datafeed = bt.feeds.PandasData(dataname=df)

    # # Initialize backtest engine
    # cerebro = bt.Cerebro()
    # cerebro.adddata(datafeed)
    # cerebro.addstrategy(RSIStrategy)
    # cerebro.broker.setcash(100000.0)  # ₹1 lakh starting capital
    # cerebro.addsizer(bt.sizers.PercentSizer, percents=10)  # Risk 10% per trade
    # cerebro.broker.setcommission(commission=0.001)  # 0.1% commission

    # # Add analyzers
    # cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    # cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    # cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    # print(f'Starting Portfolio Value: ₹{cerebro.broker.getvalue():,.2f}')
    
    # # Run backtest
    # results = cerebro.run()
    # strat = results[0]

    # print(f'Final Portfolio Value: ₹{cerebro.broker.getvalue():,.2f}')

    # # Extract performance metrics
    # trades = strat.analyzers.trades.get_analysis()
    # if trades.get('total', {}).get('total', 0) > 0:
    #     total_trades = trades.total.total
    #     won_trades = trades.won.total
    #     win_rate = won_trades / total_trades
    #     profit_factor = trades.pnl.gross.total / abs(trades.pnl.gross.loss) if trades.pnl.gross.loss != 0 else float('inf')
    # else:
    #     win_rate = 0
    #     profit_factor = 0

    # sharpe = strat.analyzers.sharpe.get_analysis().get('sharperatio', 0) or 0
    # max_dd = strat.analyzers.drawdown.get_analysis().max.drawdown or 0

    # print("\n--- BACKTEST RESULTS ---")
    # print(f"Win Rate: {win_rate:.1%}")
    # print(f"Profit Factor: {profit_factor:.2f}")
    # print(f"Sharpe Ratio: {sharpe:.2f}")
    # print(f"Max Drawdown: {max_dd:.2f}%")

    # # Optional: Plot results
    # print("\nGenerating plot...")
    # cerebro.plot(style='candlestick')


ImportError: dlopen(/Users/hemank/Documents/github/.venv/lib/python3.11/site-packages/talib/_ta_lib.cpython-311-darwin.so, 0x0002): symbol not found in flat namespace '_TA_ACCBANDS'

In [40]:
# ----------- INSTALL DEPENDENCIES --------------
# pip install backtrader yfinance pandas

import backtrader as bt
import yfinance as yf
import pandas as pd

# ======================================================================
# 1. STRATEGY DEFINITION (Must be at the top level of the script)
# ======================================================================
class TunedStrategy(bt.Strategy):
    params = dict(
        fast_ema=15,
        slow_ema=45,
        rsi_period=14,
        atr_period=14,
        rsi_long_max=65,
        rsi_short_min=35,
        atr_multiplier=2.5,
        printlog=False
    )

    def __init__(self):
        self.ema_fast = bt.ind.EMA(period=self.p.fast_ema)
        self.ema_slow = bt.ind.EMA(period=self.p.slow_ema)
        self.cross = bt.ind.CrossOver(self.ema_fast, self.ema_slow)
        self.rsi = bt.ind.RSI(period=self.p.rsi_period)
        self.atr = bt.ind.ATR(period=self.p.atr_period)
        self.order = None
        self.stop_price = None

    # NOTE: Logging and notification methods are omitted for brevity,
    # but the logic remains the same as the previous version.

    def next(self):
        if self.order: return
        if not self.position:
            long_condition = self.cross > 0 and self.rsi < self.p.rsi_long_max
            short_condition = self.cross < 0 and self.rsi > self.p.rsi_short_min
            if long_condition:
                self.stop_price = self.data.close[0] - self.p.atr_multiplier * self.atr[0]
                self.order = self.buy()
            elif short_condition:
                self.stop_price = self.data.close[0] + self.p.atr_multiplier * self.atr[0]
                self.order = self.sell()
        else:
            is_long = self.position.size > 0
            if is_long:
                new_stop = self.data.close[0] - self.p.atr_multiplier * self.atr[0]
                self.stop_price = max(self.stop_price, new_stop)
                if self.data.close[0] <= self.stop_price or self.cross < 0:
                    self.order = self.close()
            else: # Short
                new_stop = self.data.close[0] + self.p.atr_multiplier * self.atr[0]
                self.stop_price = min(self.stop_price, new_stop)
                if self.data.close[0] >= self.stop_price or self.cross > 0:
                    self.order = self.close()

# ======================================================================
# 2. MAIN EXECUTION BLOCK (Must be guarded by if __name__ == '__main__')
# ======================================================================
if __name__ == '__main__':
    # --- DATA DOWNLOAD ---
    ticker = "HDFCBANK.NS"
    df = yf.download(ticker, period="1y", interval="1h", auto_adjust=True, progress=False, multi_level_index=False)
    if df.empty:
        print(f"Error: No data downloaded for {ticker}. Exiting.")
        exit()
    datafeed = bt.feeds.PandasData(dataname=df)

    # --- OPTIMIZATION ENGINE ---
    cerebro = bt.Cerebro(optreturn=False, maxcpus=1) # Set maxcpus=1 to debug if needed

    cerebro.optstrategy(
        TunedStrategy,
        fast_ema=range(10, 26, 5),
        slow_ema=range(40, 61, 10),
        atr_multiplier=[2.0, 2.5, 3.0]
    )

    cerebro.adddata(datafeed)
    cerebro.broker.setcash(1_000_000)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=2)
    cerebro.broker.setcommission(commission=0.001)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Days, riskfreerate=0.07)

    print("Running optimization...")
    opt_results = cerebro.run()

    # --- PROCESS RESULTS ---
    final_results = []
    for run in opt_results:
        for strat in run:
            sharpe = strat.analyzers.sharpe.get_analysis().get('sharperatio', -100)
            params = strat.p
            final_results.append((sharpe, params.fast_ema, params.slow_ema, params.atr_multiplier))

    final_results.sort(key=lambda x: x[0], reverse=True)

    print("\n--- Top 10 Optimization Results (sorted by Sharpe Ratio) ---")
    print(f"{'Sharpe':<10} | {'Fast EMA':<10} | {'Slow EMA':<10} | {'ATR Multi'}")
    print("-" * 50)
    for sharpe, fast, slow, atr in final_results[:10]:
        print(f"{sharpe:<10.2f} | {fast:<10} | {slow:<10} | {atr}")


Running optimization...

--- Top 10 Optimization Results (sorted by Sharpe Ratio) ---
Sharpe     | Fast EMA   | Slow EMA   | ATR Multi
--------------------------------------------------
-1.05      | 15         | 40         | 2.0
-1.05      | 15         | 40         | 2.5
-1.05      | 15         | 40         | 3.0
-1.05      | 10         | 40         | 2.0
-1.05      | 10         | 40         | 2.5
-1.05      | 10         | 40         | 3.0
-1.05      | 10         | 50         | 2.0
-1.05      | 10         | 50         | 2.5
-1.05      | 10         | 50         | 3.0
-1.06      | 15         | 50         | 2.0


In [2]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import yfinance as yf  # For fetching real data

# Fetch historical data (example: AAPL stock from 2020-2025)
data = yf.download('AAPL', start='2020-01-01', end='2025-07-21', multi_level_index=False)

class PriceActionStrategy(Strategy):
    lookback = 10
    impulse_threshold = 0.02
    rr_threshold = 2.5

    def init(self):
        self.high = self.data.High
        self.low = self.data.Low
        self.close = self.data.Close
        self.trend = self.I(self.identify_trend)
        self.zone_type, self.zone_low, self.zone_high = self.I(self.find_zones)

    def identify_trend(self):
        trend = np.full_like(self.close, np.nan)
        valid_low = np.inf
        valid_high = -np.inf
        last_high = -np.inf
        last_low = np.inf
        trend[0] = 0  # Neutral

        for i in range(1, len(self.close)):
            current_high = self.high[i]
            current_low = self.low[i]

            if current_high > last_high:
                last_high = current_high
                if current_low < valid_low and current_high > valid_high:
                    valid_low = current_low
                    valid_high = current_high
                    trend[i] = 1  # Up
                else:
                    trend[i] = trend[i-1]
            elif current_low < last_low:
                last_low = current_low
                if current_high > valid_high and current_low < valid_low:
                    valid_high = current_high
                    valid_low = current_low
                    trend[i] = -1  # Down
                else:
                    trend[i] = trend[i-1]
            else:
                trend[i] = trend[i-1]
        return trend

    def find_zones(self):
        zone_type = np.full_like(self.close, np.nan)
        zone_low = np.full_like(self.close, np.nan)
        zone_high = np.full_like(self.close, np.nan)

        for i in range(self.lookback, len(self.close)):
            recent = self.data.Close[i-self.lookback:i]
            if abs(np.mean(np.diff(recent) / recent[:-1])) < self.impulse_threshold / 10:
                impulse = (self.close[i] - self.close[i-1]) / self.close[i-1]
                if self.trend[i] == 1 and impulse > self.impulse_threshold:
                    zone_type[i] = 1  # Demand
                    zone_low[i] = self.low[i-1]
                    zone_high[i] = self.high[i-1]
                elif self.trend[i] == -1 and impulse < -self.impulse_threshold:
                    zone_type[i] = -1  # Supply
                    zone_low[i] = self.low[i-1]
                    zone_high[i] = self.high[i-1]
        return zone_type, zone_low, zone_high

    def next(self):
        if self.zone_type[-1] == 1 and self.trend[-1] == 1:  # Demand zone in uptrend
            if self.low[-1] <= self.zone_high[-1] and self.high[-1] >= self.zone_low[-1]:
                entry = self.close[-1]
                sl = self.zone_low[-1] - 0.001 * entry
                tp = max(self.high[-10:])  # Recent high
                rr = (tp - entry) / (entry - sl)
                if rr >= self.rr_threshold:
                    self.buy(sl=sl, tp=tp)
        elif self.zone_type[-1] == -1 and self.trend[-1] == -1:  # Supply zone in downtrend
            if self.high[-1] >= self.zone_low[-1] and self.low[-1] <= self.zone_high[-1]:
                entry = self.close[-1]
                sl = self.zone_high[-1] + 0.001 * entry
                tp = min(self.low[-10:])  # Recent low
                rr = (entry - tp) / (sl - entry)
                if rr >= self.rr_threshold:
                    self.sell(sl=sl, tp=tp)

# Run the backtest
bt = Backtest(data, PriceActionStrategy, cash=10000, commission=0.002)
stats = bt.run()
print(stats)  # Outputs metrics like Win Rate, Return, Sharpe Ratio, etc.
bt.plot()  # Visualizes results


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_15427/2750859232.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download('AAPL', start='2020-01-01', end='2025-07-21', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed



Backtest.run:   0%|          | 0/1374 [00:00<?, ?bar/s]

Start                     2020-01-02 00:00:00
End                       2025-07-18 00:00:00
Duration                   2024 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]               169.28901
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
CAGR [%]                                  0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                      

GridPlot(id='p1275', ...)